<a href="https://www.advfn.com/nasdaq/nasdaq.asp?companies=A">Nasdaq Stocks</a>

In [1]:
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf

In [2]:
TODAY = (datetime.now() + timedelta(1)).date()
YEARS_OF_DATA = 10
MIN_YEARS = 3
START = TODAY - timedelta(YEARS_OF_DATA * 365)
MIN_START = TODAY - timedelta(MIN_YEARS * 365)
START, TODAY, MIN_START

(datetime.date(2012, 8, 30),
 datetime.date(2022, 8, 28),
 datetime.date(2019, 8, 29))

In [5]:
sym = (
    'afya agam agen aggr agrx agil agys agio agri rico agfy agfs ahrn '
    'alrn aimt aimd airm airt atsg airv airs abnb airg amcn ancc airn '
    'aixg akam akan song akca akba akns akla kern akro aker akli akrx '
    'akus akts akya aku  aaii alab aldn apcs alrm alsk tcrt amri afbk '
    'alac albo abdc duck aldr aldx alec alr  alrs alxn astl alco algn'
)

missing = (
    'akca aixg alxn aker alsk alac amri duck akla airm song akrx afbk '
    'aldr aaii ancc abdc aimt amcn aldn airv akns agam alab')

SYMBOLS = [s for s in sym.split() if s not in missing.split()]

In [6]:
data = (
    yf
    .download(SYMBOLS, start=START, end=TODAY)
    .rename(columns={'Adj Close': 'AdjClose'}))['AdjClose']
data.columns = SYMBOLS
data.index = pd.to_datetime(data.index)
print(data.shape)
data.head()

[*********************100%***********************]  41 of 41 completed
(2514, 41)


,afya,agen,aggr,agrx,agil,agys,agio,agri,rico,agfy,...,alrm,tcrt,albo,aldx,alec,alr,alrs,astl,alco,algn
Date,,,,,,,,,,,,,,,,,,,,,
2012-08-30,NaN,NaN,4.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,46.400002,NaN,NaN,7.355836,5.70,NaN,4.71,NaN,NaN,4.99
2012-08-31,NaN,NaN,4.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,47.299999,NaN,NaN,7.355836,5.70,NaN,4.65,NaN,NaN,4.97
2012-09-04,NaN,NaN,4.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,48.700001,NaN,NaN,7.355836,5.75,NaN,4.49,NaN,NaN,5.06
2012-09-05,NaN,NaN,4.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,47.900002,NaN,NaN,7.355836,5.75,NaN,4.46,NaN,NaN,5.33
2012-09-06,NaN,NaN,4.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49.400002,NaN,NaN,7.355836,5.75,NaN,4.47,NaN,NaN,5.39


In [7]:
str(MIN_START) in data.index

True

In [8]:
for col in data.columns:
    if np.isnan(data.loc[str(MIN_START), col]):
        data.drop(columns=[col], inplace=True)
print(data.shape)
data.head()

(2514, 24)


,agen,aggr,agrx,agri,agfy,agfs,aimd,airs,abnb,airn,...,akya,aku,apcs,alrm,tcrt,albo,aldx,alrs,astl,algn
Date,,,,,,,,,,,,,,,,,,,,,
2012-08-30,NaN,4.71,NaN,NaN,NaN,8.32,NaN,5.576311,37.310001,NaN,...,NaN,NaN,33.830002,46.400002,NaN,NaN,7.355836,4.71,NaN,4.99
2012-08-31,NaN,4.59,NaN,NaN,NaN,8.29,NaN,5.511621,37.509998,NaN,...,NaN,NaN,33.950001,47.299999,NaN,NaN,7.355836,4.65,NaN,4.97
2012-09-04,NaN,4.69,NaN,NaN,NaN,8.32,NaN,5.207576,37.669998,NaN,...,NaN,NaN,35.180000,48.700001,NaN,NaN,7.355836,4.49,NaN,5.06
2012-09-05,NaN,4.79,NaN,NaN,NaN,8.27,NaN,5.226983,37.709999,NaN,...,NaN,NaN,35.639999,47.900002,NaN,NaN,7.355836,4.46,NaN,5.33
2012-09-06,NaN,4.96,NaN,NaN,NaN,8.65,NaN,5.252859,39.209999,NaN,...,NaN,NaN,37.369999,49.400002,NaN,NaN,7.355836,4.47,NaN,5.39


In [9]:
states = pd.read_csv('../data/hmm_exp_returns.csv')
states = states[['Date', 'state']]
states.index = pd.to_datetime(states.Date)
states = states.loc[data.index]
states.drop(columns='Date', inplace=True)
states.head()

,state
Date,
2012-08-30,2
2012-08-31,2
2012-09-04,2
2012-09-05,2
2012-09-06,0


In [10]:
data = pd.concat([data, states], axis=1)

In [11]:
#plt.plot(data.aljj)

In [12]:
def get_daily_returns(x):
    idx = x.index
    n = len(x)
    x = np.array(x)
    #return x[2:n] / x[1:(n - 1)] - 1 
    returns = x[1:n] / x[0:(n-1)] - 1
    returns = pd.Series(returns, index=idx[1:])
    return returns

In [13]:
def get_sharpe(returns, window=None):
    n = len(returns)
    if window is None:
        window = n
    if len(returns) > window:
        returns = returns[(n - window):n]
    returns = returns[~np.isnan(returns)]
    sharpe = (np.sqrt(252) * returns.mean()) / returns.std()
    return sharpe

In [14]:
STATES = sorted(states.state.unique())
STATES

[0, 1, 2, 3]

In [15]:
output = []
stocks = [col for col in list(data) if col != 'state']
for stock in stocks:
    stock_data = []
    returns = get_daily_returns(data[stock])
    for state in STATES:
        sharpe_state = get_sharpe(returns[data.state == state])
        stock_data.append(sharpe_state)
    output.append(stock_data)

In [16]:
df_out = pd.DataFrame(
    output,
    columns=[f'sharpe_{state}' for state in STATES],
    index=stocks)
df_out.head()

,sharpe_0,sharpe_1,sharpe_2,sharpe_3
agen,-0.078024,0.342459,-0.204365,-0.505104
aggr,-0.964208,1.162234,1.134797,0.333312
agrx,-0.470431,-0.019382,1.022246,0.137723
agri,-1.623065,0.998961,0.512387,1.163529
agfy,-1.384595,0.990235,-0.285793,0.462183


In [17]:
big_path = '../data/nasdaq_sharpes_by_state.csv'
big_df = pd.read_csv(big_path, index_col=0)
big_df.head()

,sharpe_0,sharpe_1,sharpe_2,sharpe_3
amrk,-1.319900,1.759969,0.547862,2.297816
ascb,0.446074,-0.937235,0.669666,0.042349
amax,-1.386363,1.717022,0.570447,1.943539
az,-0.040619,0.710539,1.458525,0.507568
abgi,-1.022344,1.029165,-1.067432,1.006440


In [18]:
for stock in df_out.index:
    if stock in big_df.index:
        big_df.loc[stock, :] = big_df.loc[stock, :]
    else:
        big_df = pd.concat(
            [big_df, pd.DataFrame(df_out.loc[stock, :]).T], axis=0)
print(big_df.shape)
big_df.head()

(96, 4)


,sharpe_0,sharpe_1,sharpe_2,sharpe_3
amrk,-1.319900,1.759969,0.547862,2.297816
ascb,0.446074,-0.937235,0.669666,0.042349
amax,-1.386363,1.717022,0.570447,1.943539
az,-0.040619,0.710539,1.458525,0.507568
abgi,-1.022344,1.029165,-1.067432,1.006440


In [19]:
big_df.to_csv('../data/nasdaq_sharpes_by_state.csv', index=True)

In [20]:
def best_n_by_state(df, n, state):
    return df.sort_values(f'sharpe_{state}', ascending=False).index[:n]

In [21]:
best_0 = best_n_by_state(big_df, 10, 0)
best_1 = best_n_by_state(big_df, 10, 1)
best_2 = best_n_by_state(big_df, 10, 2)
best_3 = best_n_by_state(big_df, 10, 3)

In [22]:
list(best_0)

['aljj', 'aveo', 'avav', 'absi', 'ascb', 'adpt', 'aey', 'adn', 'gmfi', 'adex']

In [23]:
big_df.loc[best_0, 'sharpe_0']

aljj    1.424082
aveo    0.656043
avav    0.586839
absi    0.480055
ascb    0.446074
adpt    0.406870
aey     0.346758
adn     0.270009
gmfi    0.245513
adex    0.218728
Name: sharpe_0, dtype: float64